# Imports

In [93]:
import win32com.client, datetime
import pandas as pd

# Custom Functions

In [98]:
''''This function recalls all meetings for a defined number of days into the past
Meeting Subject, Duration and Responce are saved to a dictionary over the applicable
time period' '''

def getCalendarEntries(days):
    Outlook = win32com.client.Dispatch("Outlook.Application")
    ns = Outlook.GetNamespace("MAPI")
    appointments = ns.GetDefaultFolder(9).Items
    appointments.Sort("[Start]")
    appointments.IncludeRecurrences = "True"
    today = datetime.datetime.today()
    begin = today - datetime.timedelta(days=days)
    begin = begin.date().strftime("%m/%d/%Y")
    tomorrow= datetime.timedelta(days=days)+today
    end = today.date().strftime("%m/%d/%Y")
    appointments = appointments.Restrict("[Start] >= '" +begin+ "' AND [END] <= '" +end+ "'")
    events={'Subject':[],'Duration':[],'ResponseStatus':[]}
    for a in appointments:
        events['Subject'].append(str(a.Subject))
        events['Duration'].append(int(a.Duration))
        events['ResponseStatus'].append(int(a.ResponseStatus))
    return events

# Calculate proportion of working hours spent in meetings

In [ ]:
days = 5 # Specify the quantity of days you want to go back

In [99]:
events = getCalendarEntries(days)
df = pd.DataFrame.from_dict(events, orient='index').T
df = df.drop(df[df.ResponseStatus!=3].index) # Only consider meetings that have accepted
df = df.drop(df[df.Subject==''].index) # Only consider meetings that have a title

TotalMeetingHours =(df['Duration'].sum())/60
WorkingHours = days*(260/365)*8

print(TotalMeetingHours/WorkingHours)


0.2076522435897436
